In [79]:
import geopandas as gpd
from shapely.geometry import mapping
import numpy as np
import json

In [3]:
geo_data = gpd.read_file('data/ne_10m_admin_0_countries/ne_10m_admin_0_countries.shp')

In [66]:
geo_data[['CONTINENT']]

,CONTINENT
0,Asia
1,Asia
2,South America
3,South America
4,South America
...,...
253,Asia
254,Oceania
255,North America
256,North America


In [37]:
columns = [
    "ADMIN",
    "ADM0_A3",
    "CONTINENT",
    "geometry"
]

#geo_data[columns].head()
geo_data[columns][geo_data.ADMIN == 'United Kingdom']

,ADMIN,ADM0_A3,CONTINENT,geometry
78,United Kingdom,GBR,Europe,"MULTIPOLYGON (((-7.2471 55.06932, -7.25674 55...."


In [98]:
include_list = [
    "Czechia",
    "Austria"
]
remove_list = [
]

class CountryBorders:
    def __init__(self, name, coords):
        self.name = name
        self.coords = coords

country_coords = {}
# upravit tak, aby zpracoval spravne MULTIPOLYGON - zatim bere pouze nejvetsi polygon

# geo_data[(geo_data.ADMIN.isin(include_list)) & (~geo_data.ADMIN.isin(remove_list))]
for idx, row in geo_data[geo_data.CONTINENT == 'Europe'].iterrows():
    geojson_obj = mapping(row['geometry'])
    geo_coords = geojson_obj["coordinates"]
    geo_type = geojson_obj["type"]
    if geojson_obj["type"] == "Polygon":
        country_coords[row['ADMIN']] = {
            'country': row['ADMIN'],
            'country_code': geo_data[geo_data.ADMIN == row['ADMIN']]['ADM0_A3'].unique()[0],
            'continent': geo_data[geo_data.ADMIN == row['ADMIN']]['CONTINENT'].unique()[0],
            'coordinates': geo_coords[0]
        }
    else:
        i = 1
        for geo_coord in geo_coords:
            while len(geo_coord) < 4:
                geo_coord = geo_coord[0]
            
            country_coords[row['ADMIN'] + '_' + str(i).rjust(2, '0')] = {
                'country': row['ADMIN'],
                'country_code': geo_data[geo_data.ADMIN == row['ADMIN']]['ADM0_A3'].unique()[0],
                'continent': geo_data[geo_data.ADMIN == row['ADMIN']]['CONTINENT'].unique()[0],
                'coordinates': geo_coord
            }
            i += 1
    #idx = np.argmax([len(coords[0]) for coords in geo_coords])
    #country_coords[row['ADMIN']] = geo_coords[idx]
    
country_coords

{'France_01': {'country': 'France',
  'country_code': 'FRA',
  'continent': 'Europe',
  'coordinates': ((-54.1115266969999, 2.1142704430000663),
   (-54.13490799999994, 2.1106733200000605),
   (-54.14813716599994, 2.1143940230000737),
   (-54.15738724899995, 2.121654562000046),
   (-54.18854813699994, 2.1613162240001174),
   (-54.194490925999844, 2.1630732220000937),
   (-54.210045532999885, 2.161574605000098),
   (-54.229785929999935, 2.157233785000116),
   (-54.246477416999966, 2.151601054000082),
   (-54.263789021999884, 2.148190410000055),
   (-54.31401851399991, 2.1548566690000683),
   (-54.32285518499992, 2.1573371380000452),
   (-54.33505082199994, 2.163900045000034),
   (-54.360940714999884, 2.188187968000122),
   (-54.37391149899989, 2.196507873000087),
   (-54.38052608299998, 2.197696432000015),
   (-54.38781245999988, 2.1963011680000335),
   (-54.40254024299989, 2.197127991000059),
   (-54.4114802659999, 2.1997118120000607),
   (-54.42708654899994, 2.2071532190000767),
   (-

In [10]:
class Country:
    def __init__(self, country, country_code, continent, coordinates):
        self.country = country
        self.country_code = country_code
        self.continent = continent
        self.coordinates = coordinates

In [99]:
with open('data/countries.json', 'w') as outfile:
    json.dump(country_coords, outfile)

In [90]:
countries = {}

for idx, row in geo_data.iterrows():
    geojson_obj = mapping(row['geometry'])
    geo_coords = geojson_obj["coordinates"]
    geo_type = geojson_obj["type"]
    if geo_type == "Polygon":
        countries[row['ADMIN']] = {
            'country': row['ADMIN'],
            'country_code': geo_data[geo_data.ADMIN == row['ADMIN']]['ADM0_A3'].unique()[0],
            'continent': geo_data[geo_data.ADMIN == row['ADMIN']]['CONTINENT'].unique()[0],
            'coordinates': geo_coords[0]
        }
    else:
        i = 1
        for geo_coord in geo_coords:
            while len(geo_coord) < 4:
                geo_coord = geo_coord[0]
            countries[row['ADMIN'] + '_' + str(i).rjust(2, '0')] = {
                'country': row['ADMIN'],
                'country_code': geo_data[geo_data.ADMIN == row['ADMIN']]['ADM0_A3'].unique()[0],
                'continent': geo_data[geo_data.ADMIN == row['ADMIN']]['CONTINENT'].unique()[0],
                'coordinates': geo_coord
            }
            i += 1
    
with open('data/countries.json', 'w') as outfile:
    json.dump(countries, outfile)